L'objectif de ce notebook est la création d'un notebook portant sur l'analyse de l'évolution des thèmes abordés au cours des JT français.

Vincent Lucas, 01/2025

# Imports

In [1]:
!pip install streamlit
!pip install gradio
!pip install squarify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
from google.colab import drive
import matplotlib.pyplot as plt
import os
import pandas as pd
import gradio as gr
import io
import seaborn as sns
import squarify
import plotly.express as px


# Connexion au drive

In [3]:
drive.mount("/content/drive", force_remount=True)
chemin_donnees = "/content/drive/MyDrive/PIP2025/Donnees"

Mounted at /content/drive


# Chargement des données

In [4]:
path = os.path.join(chemin_donnees, "Classement thématique des sujets de journeaux télévisés", "ina-barometre-jt-tv-donnees-quotidiennes-2000-2020-nbre-sujets-durees-202410.csv")
df = pd.read_csv(
    path,
    encoding='latin-1',
    sep=';',
    header=None,
    names=["Date", "Chaine", "Vide", "Theme", "NbEmissions", "Temps"]
)
df.dropna(axis=1, how='all', inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Annee'] = df['Date'].dt.year.astype(int)
df

,Date,Chaine,Theme,NbEmissions,Temps,Annee
0,2000-01-02,France 3,Catastrophes,2,235,2000
1,2000-01-02,France 3,Culture-loisirs,1,138,2000
2,2000-01-02,France 3,Environnement,3,306,2000
3,2000-01-02,France 3,International,2,241,2000
4,2000-01-02,France 3,Société,2,160,2000
...,...,...,...,...,...,...
268419,2020-12-31,M6,Histoire-hommages,1,109,2020
268420,2020-12-31,M6,International,3,123,2020
268421,2020-12-31,M6,Politique France,1,46,2020
268422,2020-12-31,M6,Santé,3,96,2020


On génère le pourcentage de temps

In [5]:
# Calculer le temps total par chaîne
df['Temps_Total_Chaine_Journee'] = df.groupby(['Date', 'Chaine'])['Temps'].transform('sum')

# Calculer le pourcentage du temps pour chaque thème par chaîne
df['Temps_Pourcentage'] = (df['Temps'] / df['Temps_Total_Chaine_Journee']) * 100

# Optionnel : Arrondir les pourcentages à deux décimales
df['Temps_Pourcentage'] = df['Temps_Pourcentage'].round(2)

df.head(50)

,Date,Chaine,Theme,NbEmissions,Temps,Annee,Temps_Total_Chaine_Journee,Temps_Pourcentage
0,2000-01-02,France 3,Catastrophes,2,235,2000,1254,18.74
1,2000-01-02,France 3,Culture-loisirs,1,138,2000,1254,11.00
2,2000-01-02,France 3,Environnement,3,306,2000,1254,24.40
3,2000-01-02,France 3,International,2,241,2000,1254,19.22
4,2000-01-02,France 3,Société,2,160,2000,1254,12.76
5,2000-01-02,France 3,Sport,2,174,2000,1254,13.88
6,2000-01-03,France 3,Catastrophes,3,329,2000,1190,27.65
7,2000-01-03,France 3,Culture-loisirs,1,38,2000,1190,3.19
8,2000-01-03,France 3,Economie,2,191,2000,1190,16.05
9,2000-01-03,France 3,Education,2,224,2000,1190,18.82


In [6]:
# Calculer le temps moyen en pourcentage par thème et par chaîne
df_moyenne = df.groupby(['Chaine', 'Theme'])['Temps_Pourcentage'].mean().reset_index()

df_moyenne['Temps_Pourcentage_Moyen'] = df_moyenne['Temps_Pourcentage'].round(2)
df_moyenne.head(50)

,Chaine,Theme,Temps_Pourcentage,Temps_Pourcentage_Moyen
0,Arte,Catastrophes,10.262847,10.26
1,Arte,Culture-loisirs,19.274748,19.27
2,Arte,Economie,15.867537,15.87
3,Arte,Education,14.661704,14.66
4,Arte,Environnement,13.680477,13.68
5,Arte,Faits divers,9.389227,9.39
6,Arte,Histoire-hommages,13.215509,13.22
7,Arte,International,45.644500,45.64
8,Arte,Justice,11.647704,11.65
9,Arte,Politique France,14.620030,14.62


# Dashboard V1

In [7]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date_ordinal'] = df['Date'].map(pd.Timestamp.toordinal)


# Calculer le temps total en heures par chaîne
df_total = df.groupby('Chaine')['Temps'].sum().reset_index()
df_total['Temps_heures'] = (df_total['Temps'] / 3600).round(2)  # Convertir en heures et arrondir

chaines = df['Chaine'].unique().tolist()
themes = df['Theme'].unique().tolist()

# Préparer les données pour le graphique en ligne
df_line = df.groupby(['Annee', 'Theme'])['Temps'].sum().reset_index()
df_line['Temps_heures'] = (df_line['Temps'] / 3600).round(2)  # Convertir en heures et arrondir


In [8]:
def create_pie(selected_chaines, selected_themes, start_year, end_year):
    # Filtrer les données en fonction de la plage d'années
    mask = (df['Annee'] >= start_year) & (df['Annee'] <= end_year)
    df_filtered = df[mask]

    # Filtrer par thèmes s'il y en a de sélectionnés
    if selected_themes:
        df_filtered = df_filtered[df_filtered['Theme'].isin(selected_themes)]

    # Filtrer par chaînes si une sélection est faite, sinon garder toutes
    if selected_chaines:
        df_filtered = df_filtered[df_filtered['Chaine'].isin(selected_chaines)]

    # Calculer le temps total en secondes par chaîne, puis le convertir en heures
    df_total = df_filtered.groupby('Chaine')['Temps'].sum().reset_index()
    df_total['Temps_heures'] = (df_total['Temps'] / 3600).round(2)

    # Créer le diagramme camembert
    fig = px.pie(
        df_total,
        names='Chaine',
        values='Temps_heures',
        title="Temps d'émission par chaîne (en heures)",
        hole=0.3
    )
    fig.update_traces(textposition='inside', textinfo='percent+label')

    return fig


In [9]:
def create_line_chart(selected_chaines, selected_themes, start_year, end_year):
    # Filtrer d'abord par année
    mask = (df['Annee'] >= start_year) & (df['Annee'] <= end_year)
    df_filtered = df[mask]

    # Appliquer le filtre sur les chaînes (si des chaînes sont sélectionnées)
    if selected_chaines:
        df_filtered = df_filtered[df_filtered['Chaine'].isin(selected_chaines)]

    # Regrouper les données par Annee et Theme
    df_line = df_filtered.groupby(['Annee', 'Theme'])['Temps'].sum().reset_index()
    df_line['Temps_heures'] = (df_line['Temps'] / 3600).round(2)  # Conversion en heures

    # Appliquer le filtre sur les thèmes (si sélectionnés)
    if selected_themes:
        df_line = df_line[df_line['Theme'].isin(selected_themes)]

    # Créer le graphique en ligne
    fig = px.line(
        df_line,
        x='Annee',
        y='Temps_heures',
        color='Theme',
        markers=True,
        title="Évolution des Thèmes Traités par Année (en heures)"
    )
    fig.update_layout(xaxis_title='Année', yaxis_title='Temps Total d\'Émission (heures)')
    return fig


In [10]:
# Fonction pour générer le graphique en bulles avec filtrage par année
def create_bubble_chart(selected_chaines, selected_themes, start_year, end_year):
    # Filtrer les données en fonction des années sélectionnées
    mask = (df['Annee'] >= start_year) & (df['Annee'] <= end_year)
    df_filtered = df[mask]

    # Calculer le temps moyen en pourcentage par thème et par chaîne
    df_moyenne = df_filtered.groupby(['Chaine', 'Theme'])['Temps_Pourcentage'].mean().reset_index()
    df_moyenne['Temps_Pourcentage_Moyen'] = df_moyenne['Temps_Pourcentage'].round(2)

    # Filtrer les données en fonction des chaînes sélectionnées
    if not selected_chaines:
        filtered_df = df_moyenne
    else:
        filtered_df = df_moyenne[df_moyenne['Chaine'].isin(selected_chaines)]

    # Filtrer les données en fonction des thèmes sélectionnés
    if not selected_themes:
        # Si aucun thème n'est sélectionné, afficher tous les thèmes
        pass  # Aucun filtrage supplémentaire nécessaire
    else:
        filtered_df = filtered_df[filtered_df['Theme'].isin(selected_themes)]

    # Créer le graphique en bulles
    fig = px.scatter(
        filtered_df,
        x='Chaine',
        y='Theme',
        size='Temps_Pourcentage_Moyen',
        color='Theme',
        hover_name='Theme',
        size_max=60,
        title="Thèmes les Plus Abordés par Chaîne (Temps Moyen d'Émission)"
    )

    # Ajustements de la mise en page pour espacer les thèmes et agrandir le visuel
    fig.update_layout(
        height=800,  # Augmenter la hauteur du graphique
        xaxis_title='Chaîne',
        yaxis_title='Thème',
        legend_title='Thème',
        title_x=0.5,  # Centrer le titre
        yaxis=dict(
            automargin=True,
            tickmode='array',
            tickfont=dict(size=12),  # Augmenter la taille de la police des ticks
        ),
        margin=dict(l=100, r=100, t=100, b=100),  # Ajouter des marges pour éviter le chevauchement
    )

    # Optionnel : Ajuster l'espacement entre les catégories sur l'axe Y
    unique_themes = sorted(filtered_df['Theme'].unique())
    fig.update_yaxes(categoryorder='array', categoryarray=unique_themes)

    return fig


In [11]:
import itertools
from sklearn.metrics.pairwise import cosine_similarity

def create_top5_similarity(selected_chaines, selected_themes, start_year, end_year):
    # Filtrer les données en fonction des années sélectionnées
    df_filtered = df[(df['Annee'] >= start_year) & (df['Annee'] <= end_year)]

    # Filtrer par chaînes sélectionnées, si des chaînes sont choisies
    if selected_chaines:
        df_filtered = df_filtered[df_filtered['Chaine'].isin(selected_chaines)]

    # Filtrer par thèmes sélectionnés, si des thèmes sont choisis
    if selected_themes:
        df_filtered = df_filtered[df_filtered['Theme'].isin(selected_themes)]

    # Vérifier qu'il y a suffisamment de données après le filtrage
    if df_filtered.empty:
        return pd.DataFrame(columns=["Chaîne 1", "Chaîne 2", "Cosine Similarity"])

    # Calculer les proportions d'émissions par thème pour chaque chaîne
    proportions = df_filtered.groupby(['Chaine', 'Theme'])['Temps_Pourcentage'].sum().reset_index()
    proportions_total = proportions.groupby('Chaine')['Temps_Pourcentage'].sum().reset_index().rename(columns={'Temps_Pourcentage': 'Total_Temps_Pourcentage'})
    proportions = proportions.merge(proportions_total, on='Chaine')
    proportions['Proportion'] = proportions['Temps_Pourcentage'] / proportions['Total_Temps_Pourcentage']

    # Créer un pivot table pour avoir les proportions par thème en colonnes
    pivot = proportions.pivot(index='Chaine', columns='Theme', values='Proportion').fillna(0)

    # Calculer la Cosine Similarity entre chaque paire de chaînes
    similarity_matrix = cosine_similarity(pivot)
    similarity_df = pd.DataFrame(similarity_matrix, index=pivot.index, columns=pivot.index)

    # Générer toutes les paires de chaînes sans duplication
    pairs = list(itertools.combinations(pivot.index, 2))
    similarity_list = []

    for chain1, chain2 in pairs:
        similarity = similarity_df.loc[chain1, chain2]
        similarity_list.append({
            'Chaîne 1': chain1,
            'Chaîne 2': chain2,
            'Cosine Similarity': round(similarity, 4)
        })

    similarity_df_final = pd.DataFrame(similarity_list)

    # Trier par Cosine Similarity décroissante et prendre le top 5
    top5 = similarity_df_final.sort_values(by='Cosine Similarity', ascending=False).head(5).reset_index(drop=True)

    return top5


In [12]:
# Définir l'interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Analyse thématique des sujets de journeaux télévisés")

    # Première ligne : Sliders de sélection des années
    with gr.Row():
        start_year_slider = gr.Slider(
            minimum=df['Annee'].min(),
            maximum=df['Annee'].max(),
            step=1,
            label="Année de début",
            value=df['Annee'].min()
        )
        end_year_slider = gr.Slider(
            minimum=df['Annee'].min(),
            maximum=df['Annee'].max(),
            step=1,
            label="Année de fin",
            value=df['Annee'].max()
        )

    # Deuxième ligne : Filtre des chaînes
    with gr.Row():
        chaine_selection = gr.CheckboxGroup(
            label="Sélectionnez les chaînes",
            choices=chaines,
            value=chaines  # Par défaut, toutes les chaînes sont sélectionnées
        )

    # Troisième ligne : Filtre des thèmes
    with gr.Row():
        theme_selection = gr.CheckboxGroup(
            label="Sélectionnez les thèmes",
            choices=themes,
            value=themes  # Par défaut, tous les thèmes sont sélectionnés
        )

    # Quatrième ligne : Les deux visuels côte à côte
    with gr.Row():
        pie_chart = gr.Plot(value=create_pie(chaines, themes, start_year_slider.value, end_year_slider.value))
        line_chart = gr.Plot(value=create_line_chart(chaines, themes, start_year_slider.value, end_year_slider.value))

    # Cinquième ligne : Bubble Chart
    with gr.Row():
        bubble_chart = gr.Plot(value=create_bubble_chart(chaines, themes, start_year_slider.value, end_year_slider.value))

    with gr.Row():
        top5_table = gr.Dataframe(
            headers=["Chaîne 1", "Chaîne 2", "Cosinus similarity"],
            label="Top 5 des Ressemblances entre Chaînes",
            interactive=False,
            value=create_top5_similarity(chaines, themes, start_year_slider.value, end_year_slider.value)
        )

    # Lier les filtres et les sliders aux visuels existants
    chaine_selection.change(
      fn=create_pie,
      inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
      outputs=pie_chart
    )

    theme_selection.change(
        fn=create_pie,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=pie_chart
    )

    start_year_slider.change(
        fn=create_pie,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=pie_chart
    )

    end_year_slider.change(
        fn=create_pie,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=pie_chart
    )

    # Liaison pour le graphique en ligne
    chaine_selection.change(
        fn=create_line_chart,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=line_chart
    )

    theme_selection.change(
        fn=create_line_chart,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=line_chart
    )

    start_year_slider.change(
        fn=create_line_chart,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=line_chart
    )

    end_year_slider.change(
        fn=create_line_chart,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=line_chart
    )

    # Liaison pour le Bubble Chart
    chaine_selection.change(
        fn=create_bubble_chart,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=bubble_chart
    )

    theme_selection.change(
        fn=create_bubble_chart,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=bubble_chart
    )

    start_year_slider.change(
        fn=create_bubble_chart,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=bubble_chart
    )

    end_year_slider.change(
        fn=create_bubble_chart,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=bubble_chart
    )

    chaine_selection.change(
        fn=create_top5_similarity,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=top5_table
    )

    theme_selection.change(
        fn=create_top5_similarity,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=top5_table
    )

    start_year_slider.change(
        fn=create_top5_similarity,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=top5_table
    )

    end_year_slider.change(
        fn=create_top5_similarity,
        inputs=[chaine_selection, theme_selection, start_year_slider, end_year_slider],
        outputs=top5_table
    )

    gr.Markdown("""
    **Instructions :**
    - Utilisez les sliders de sélection des années ci-dessus pour définir la plage de dates des données à visualiser.
    - Utilisez les cases à cocher suivantes pour filtrer les chaînes et les thèmes que vous souhaitez visualiser.
    - Le diagramme camembert affiche le temps d'émission par chaîne sélectionnée dans la plage de dates définie.
    - Le graphique en ligne montre l'évolution des thèmes sélectionnés au fil des années en termes de temps total d'émission dans la plage de dates définie.
    - Le graphique en bulles représente les thèmes les plus abordés par chaîne avec le temps moyen d'émissions dans la plage de dates définie.
    - Le tableau "Top 5 des Ressemblances entre Chaînes" affiche les paires de chaînes les plus similaires basées sur la MSE de leurs proportions d'émissions par thème.
    """)

# Lancer l'interface
demo.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://119dd374708565f946.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
import plotly.express as px


df_cat = df[df['Theme'] == "Catastrophes"].copy()

df_cat_line = df_cat.groupby('Annee')['Temps'].sum().reset_index()
df_cat_line['Temps_heures'] = (df_cat_line['Temps'] / 3600).round(2)


fig = px.line(
    df_cat_line,
    x='Annee',
    y='Temps_heures',
    markers=True,
    title="Évolution de la couverture médiatique par thème 'Catastrophes' chaque Année"
)

fig.update_layout(
    xaxis_title='Année',
    yaxis_title='Temps Total d\'Émission (heures)',
    template='plotly_white'
)

events = [
    {
        "annee": 2001,
        "label": "Séisme du Gujarat (26/01/2001, Inde)",
        "victimes": "≈20 000"
    },
    {
        "annee": 2005,
        "label": "Séisme du Cachemire (8/10/2005, Pakistan)",
        "victimes": "≈80 000"
    },
    {
        "annee": 2008,
        "label": "Cyclone Nargis (Birmanie)",
        "victimes": "≈140 000"
    },
    {
        "annee": 2010,
        "label": "Séisme en Haïti (12/01/2010)",
        "victimes": "≈280 000"
    },
    {
        "annee": 2014,
        "label": "Typhon Rammasun (Philippines/Chine/Vietnam)",
        "victimes": "≈225"
    },
    {
        "annee": 2018,
        "label": "Séisme & Tsunami de Sulawesi (Indonésie)",
        "victimes": "≈4 300"
    }
]

# 6) Ajouter les annotations pour chaque année ciblée
for evt in events:
    annee_evt = evt["annee"]
    df_event = df_cat_line[df_cat_line['Annee'] == annee_evt]
    if not df_event.empty:
        # Récupère la valeur de Temps_heures pour l'année concernée
        y_value = df_event['Temps_heures'].iloc[0]

        fig.add_annotation(
        x=annee_evt,
        y=y_value,
        text=f"<b>{evt['label']}</b><br>{evt['victimes']} victimes",
        showarrow=True,
        arrowhead=2,
        arrowcolor="red",
        ax=30,   # Décalage sur l'axe X pour la flèche
        ay=-40,  # Décalage sur l'axe Y
        bgcolor="rgba(255, 255, 255, 0.7)"
        )



        # (Optionnel) Ligne verticale pointillée pour marquer l'année
        fig.add_shape(
            type="line",
            x0=annee_evt,
            x1=annee_evt,
            y0=0,
            y1=y_value,
            line=dict(color="red", dash="dot")
        )

# 7) Afficher la figure
fig.show()